In [33]:
import paramiko
 
# SSH connection settings
host = 'node1'  # Replace 'node1' with your actual hostname or IP address
username = 'nishanths'  # Replace 'your_username' with your actual username
password = 'newuser123'  # Replace 'your_password' with your actual password
 
# Establish SSH connection
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
 
print(torch.cuda.is_available())
ssh_client.connect(hostname=host, username=username, password=password)
print(torch.cuda.is_available())
print("SSH connection established.")
    
# finally:
#     # Close the SSH connection
#     ssh_client.close()
#     print("SSH connection closed.")

False
False
SSH connection established.


In [28]:
import os
import json
import random
from tqdm import tqdm

import torch
from transformers import AutoTokenizer, AutoModel

#### Set up OpenAI API

In [29]:
# import openai

# openai.api_key = os.getenv("OPENAI_API_KEY")

# def call_gpt(cur_prompt, stop):
#     ans = openai.Completion.create(
#                 model="text-davinci-003",
#                 max_tokens=256,
#                 stop=stop,
#                 prompt=cur_prompt,
#                 temperature=0)
#     returned = ans['choices'][0]['text']
#     return returned

#### Functions for retrieval models (Contriever)

In [30]:
def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings

def get_sent_embeddings(sents, contriever, tok, BSZ=32):    
    all_embs = []
    for i in tqdm(range(0, len(sents), BSZ)):
        sent_batch = sents[i:i+BSZ]
        inputs = tok(sent_batch, padding=True, truncation=True, return_tensors='pt').to("cuda")
        with torch.no_grad():
            outputs = contriever(**inputs)
            embeddings = mean_pooling(outputs[0], inputs['attention_mask'])
        all_embs.append(embeddings.cpu())
    all_embs = torch.vstack(all_embs)
    return all_embs

def retrieve_facts(query, fact_embs, contriever, tok, k=1):
    inputs = tok([query], padding=True, truncation=True, return_tensors='pt').to("cuda")
    with torch.no_grad():
        outputs = contriever(**inputs)
        query_emb = mean_pooling(outputs[0], inputs['attention_mask']).cpu()
    sim = (query_emb @ fact_embs.T)[0]
    knn = sim.topk(k, largest=True)
    return knn.indices

#### Load dataset

In [31]:
with open('datasets/MQuAKE-CF-3k.json', 'r') as f:
    dataset = json.load(f)

#### Build a memory index which contains all the edits

In [32]:
new_facts = set()
for d in dataset:
    for r in d["requested_rewrite"]:
        new_facts.add(f'{r["prompt"].format(r["subject"])} {r["target_new"]["str"]}')
new_facts = list(new_facts)

In [24]:
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")
print(device)
ontriever = AutoModel.from_pretrained("facebook/contriever-msmarco").to(device)
tokenizer = AutoTokenizer.from_pretrained("facebook/contriever-msmarco")

cpu


KeyboardInterrupt: 

In [7]:
embs = get_sent_embeddings(new_facts, contriever, tokenizer)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:01<00:00, 46.75it/s]


In [8]:
# Run test for retrieval index
fact_ids = retrieve_facts("Who is the president of the US?", embs, contriever, tokenizer)
print(new_facts[fact_ids[0]])

The name of the current head of state in United States of America is Norodom Sihamoni


In [1]:
import torch
from transformers import GPTJForCausalLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", revision="float16", low_cpu_mem_usage=True)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

/home/nishanths/miniconda3/envs/mquake/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import StoppingCriteria
class MyStoppingCriteria(StoppingCriteria):
    def __init__(self, target_sequence, prompt):
        self.target_sequence = target_sequence
        self.prompt=prompt

    def __call__(self, input_ids, scores, **kwargs):
        # Get the generated text as a string
        generated_text = tokenizer.decode(input_ids[0])
        generated_text = generated_text.replace(self.prompt,'')
        # Check if the target sequence appears in the generated text
        if self.target_sequence in generated_text:
            return True  # Stop generation

        return False  # Continue generation

    def __len__(self):
        return 1

    def __iter__(self):
        yield self

In [13]:
def call_gptj(curr_prompt, stop):
    input_ids = tokenizer(curr_prompt, return_tensors="pt").input_ids.to(device)
    out = model.generate(
                input_ids,
                do_sample=True,
                temperature=0.9,
                max_new_tokens=256,
                stopping_criteria=MyStoppingCriteria(stop, curr_prompt))
    return out

#### Run MeLLo

In [14]:
# read prompts
with open('prompts/MeLLo-prompt.txt', 'r') as f:
    task_prompt = f.read()
stop = "Retrieved fact:"

In [15]:
# Run MeLLo on the first T (T=10) examples
T = 10

cor = 0
tot = 0

for d in tqdm(dataset[:10]):
    tot += 1
    for q in d["questions"]:
        found_ans = False
        prompt = task_prompt + "\n\nQustion: " + q
        for i in range(4):
            # prompt the model to generate a subquestion and a tentative answer
            gen = call_gptj(prompt, stop)
            last_sent = gen.strip().split('\n')[-1]
            
            # if final answer is there, get the answer and exit
            if last_sent.startswith('Final answer: '):
                found_ans = True
                ans = last_sent[len("Final answer: "):]
                break
            
            # otherwise, extract the generated subquestion
            if len(gen.strip().split('\n')) < 2:
                break # failed case
            subquestion = gen.strip().split('\n')[-2]
            if not subquestion.startswith('Subquestion: '):
                break # failed case
            subquestion = subquestion[len("Subquestion: "):]
            
            # retrieve an edited fact using the generated subquestion
            fact_ids = retrieve_facts(subquestion, embs, contriever, tokenizer)
            fact_sent = new_facts[fact_ids[0]]
            
            # put the retrieved fact at the end of the prompt, the model self-checks if it contradicts
            prompt = prompt + gen + 'Retrieved fact: ' + fact_sent + '.'
            
        prompt = prompt + gen
        
        if not found_ans:
            continue
        # if the answer is correct
        if ans == d["new_answer"] or ans in d["new_answer_alias"]:
            cor += 1
            break

print(f'Multi-hop acc = {cor / tot} ({cor} / {tot})')

  0%|          | 0/10 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  0%|          | 0/10 [01:39<?, ?it/s]


AttributeError: 'Tensor' object has no attribute 'strip'